In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import numpy as np

In [2]:
power_ratings = pd.read_csv('power_ratings.csv')
kpower = pd.read_csv('kpower.csv', encoding='latin-1')
kp_ratings = pd.merge(kpower,power_ratings)

In [3]:
kp_ratings['song'] = kp_ratings['title'] +' - '+ kp_ratings['artist']
print(kp_ratings.shape)
kp_ratings.head()

(3193, 7)


,song_id,title,artist,tempo,user_id,rating,song
0,1,DUMDi DUMDi,(G)I-DLE,123.047,1,5,DUMDi DUMDi - (G)I-DLE
1,1,DUMDi DUMDi,(G)I-DLE,123.047,6,2,DUMDi DUMDi - (G)I-DLE
2,1,DUMDi DUMDi,(G)I-DLE,123.047,8,2,DUMDi DUMDi - (G)I-DLE
3,1,DUMDi DUMDi,(G)I-DLE,123.047,28,1,DUMDi DUMDi - (G)I-DLE
4,1,DUMDi DUMDi,(G)I-DLE,123.047,65,2,DUMDi DUMDi - (G)I-DLE


In [4]:
numpower_rating = kp_ratings.groupby('song')['rating'].count().reset_index()

In [5]:
numpower_rating.head()

,song,rating
0,4 Walls - f(x),19
1,4x4ever - MAMAMOO,17
2,9 and Three Quarters (Run Away) - TOMORROW X T...,12
3,A - RAINBOW,16
4,AHH OOP 2021 - MAMAMOO,16


In [6]:
numpower_rating.rename(columns={'rating':'no of ratings'}, inplace=True)
final_power = kp_ratings.merge(numpower_rating, on='song')
final_power

,song_id,title,artist,tempo,user_id,rating,song,no of ratings
0,1,DUMDi DUMDi,(G)I-DLE,123.047,1,5,DUMDi DUMDi - (G)I-DLE,20
1,1,DUMDi DUMDi,(G)I-DLE,123.047,6,2,DUMDi DUMDi - (G)I-DLE,20
2,1,DUMDi DUMDi,(G)I-DLE,123.047,8,2,DUMDi DUMDi - (G)I-DLE,20
3,1,DUMDi DUMDi,(G)I-DLE,123.047,28,1,DUMDi DUMDi - (G)I-DLE,20
4,1,DUMDi DUMDi,(G)I-DLE,123.047,65,2,DUMDi DUMDi - (G)I-DLE,20
...,...,...,...,...,...,...,...,...
3188,632,Dancing King,Yoo Jae Seok,127.987,216,1,Dancing King - Yoo Jae Seok,19
3189,632,Dancing King,Yoo Jae Seok,127.987,237,3,Dancing King - Yoo Jae Seok,19
3190,632,Dancing King,Yoo Jae Seok,127.987,281,1,Dancing King - Yoo Jae Seok,19
3191,632,Dancing King,Yoo Jae Seok,127.987,316,0,Dancing King - Yoo Jae Seok,19


In [7]:
final_power = final_power[final_power['no of ratings']>=5] #only songs above 5 users have rated is qualified, to avoid noise
final_power.shape

(3189, 8)

In [8]:
final_power.drop_duplicates(['user_id','song'], inplace=True)
final_power.shape

<ipython-input-8-c08c74579dae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_power.drop_duplicates(['user_id','song'], inplace=True)


(3102, 8)

In [9]:
power_pivot = final_power.pivot_table(columns='user_id', index='song', values='rating')
power_pivot.fillna(0, inplace=True)

In [10]:
print(power_pivot.shape)
power_pivot

(207, 332)


user_id,1,2,3,4,5,6,7,8,9,10,...,323,324,325,326,327,328,329,330,331,332
song,,,,,,,,,,,,,,,,,,,,,
4 Walls - f(x),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4x4ever - MAMAMOO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9 and Three Quarters (Run Away) - TOMORROW X TOGETHER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
A - RAINBOW,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AHH OOP 2021 - MAMAMOO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X1-MA - PRODUCE X 101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
YES or YES - TWICE,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YOU&I - IU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
from scipy.sparse import csr_matrix
power_sparse = csr_matrix(power_pivot)
type(power_sparse)

scipy.sparse.csr.csr_matrix

In [12]:
from sklearn.neighbors import NearestNeighbors
power_model = NearestNeighbors(algorithm='brute')

In [13]:
power_model.fit(power_sparse)

NearestNeighbors(algorithm='brute')

In [14]:
distances, suggestions = power_model.kneighbors(power_pivot.iloc[4,:].values.reshape(1,-1), n_neighbors=15)

In [15]:
suggestions

array([[  4, 120,  11, 135, 151,  67, 100, 199, 129,  44, 147,  76, 171,
         38, 154]], dtype=int64)

In [16]:
for i in range(len(suggestions)):
    print(power_pivot.index[suggestions[i]])

Index(['AHH OOP 2021 - MAMAMOO', 'Mirror Mirror - 4Minute',
       'Am I too easy? - U-KISS', 'PTT (Paint The Town) - LOONA',
       'SHA LA LA - PENTAGON', 'FANCY - TWICE', 'In the morning - ITZY',
       'With U (Feat. Chungha) - Samuel', 'Nonstop - OH MY GIRL',
       'Crazy - TEEN TOP', 'Rose - LeeHi', 'Fly - GOT7', 'Stay Gold - BTS',
       'CALL CALL CALL! - SEVENTEEN', 'Save - NCT 127'],
      dtype='object', name='song')


In [17]:
power_pivot.index[50]

'DIVE - Jinyoung'

In [18]:
np.where(power_pivot.index=='DIVE - Jinyoung')[0][0]

50

In [19]:
def recommend_kmusic_power(song):
    song_id=np.where(power_pivot.index==song)[0][0]
    distances, suggestions = power_model.kneighbors(power_pivot.iloc[song_id,:].values.reshape(1,-1), n_neighbors=15)
    
    for i in range(len(suggestions)):
        print(power_pivot.index[suggestions[i]])

In [20]:
recommend_kmusic_power('Nonstop - OH MY GIRL')

Index(['Nonstop - OH MY GIRL', 'Mirror Mirror - 4Minute',
       'PTT (Paint The Town) - LOONA', 'FANCY - TWICE', 'Crazy - TEEN TOP',
       'Why Why Why - iKON', 'In the morning - ITZY',
       'Feel Good (SECRET CODE) - fromis_9', 'Russian Roulette - Red Velvet',
       'Fly - GOT7', 'ALIEN - LEE SUHYUN', 'Power - EXO',
       'Hot Issue - 4Minute', 'Ready to love - SEVENTEEN', 'Save Me - BTS'],
      dtype='object', name='song')


In [21]:
ratepower = kp_ratings.groupby('song_id')['rating'].count().reset_index()
ratepower

,song_id,rating
0,1,20
1,8,17
2,9,17
3,15,20
4,16,10
...,...,...
203,615,16
204,626,21
205,627,15
206,630,18


In [22]:
power_list = pd.merge(kpower,ratepower)
power_list.rename(columns={'rating':'no of ratings'}, inplace=True)
power_list 

,song_id,title,artist,tempo,no of ratings
0,1,DUMDi DUMDi,(G)I-DLE,123.047,20
1,8,Spotlight,1THE9,136.014,17
2,9,I Am The Best,2NE1,127.997,17
3,15,Heart To Heart,4Minute,129.961,20
4,16,Hot Issue,4Minute,128.008,10
...,...,...,...,...,...
203,615,Crush,Weki Meki,124.007,16
204,626,UNNATURAL,WJSN,124.203,21
205,627,So Hot,Wonder Girls,139.985,15
206,630,FLASH,X1,132.040,18


In [23]:
power_list['song'] = power_list['title'] +' - '+ power_list['artist']
power_list

,song_id,title,artist,tempo,no of ratings,song
0,1,DUMDi DUMDi,(G)I-DLE,123.047,20,DUMDi DUMDi - (G)I-DLE
1,8,Spotlight,1THE9,136.014,17,Spotlight - 1THE9
2,9,I Am The Best,2NE1,127.997,17,I Am The Best - 2NE1
3,15,Heart To Heart,4Minute,129.961,20,Heart To Heart - 4Minute
4,16,Hot Issue,4Minute,128.008,10,Hot Issue - 4Minute
...,...,...,...,...,...,...
203,615,Crush,Weki Meki,124.007,16,Crush - Weki Meki
204,626,UNNATURAL,WJSN,124.203,21,UNNATURAL - WJSN
205,627,So Hot,Wonder Girls,139.985,15,So Hot - Wonder Girls
206,630,FLASH,X1,132.040,18,FLASH - X1


In [24]:
import pickle

In [26]:
pickle.dump(power_list,open('kpower.pkl','wb'))

In [25]:
power_list['song']

0           DUMDi DUMDi - (G)I-DLE
1                Spotlight - 1THE9
2             I Am The Best - 2NE1
3         Heart To Heart - 4Minute
4              Hot Issue - 4Minute
                  ...             
203              Crush - Weki Meki
204               UNNATURAL - WJSN
205          So Hot - Wonder Girls
206                     FLASH - X1
207    Dancing King - Yoo Jae Seok
Name: song, Length: 208, dtype: object

In [27]:
power_list.to_dict()

{'song_id': {0: 1,
  1: 8,
  2: 9,
  3: 15,
  4: 16,
  5: 17,
  6: 18,
  7: 19,
  8: 20,
  9: 27,
  10: 28,
  11: 32,
  12: 33,
  13: 35,
  14: 39,
  15: 40,
  16: 41,
  17: 56,
  18: 61,
  19: 64,
  20: 67,
  21: 69,
  22: 70,
  23: 71,
  24: 72,
  25: 73,
  26: 75,
  27: 78,
  28: 79,
  29: 80,
  30: 81,
  31: 82,
  32: 84,
  33: 87,
  34: 89,
  35: 90,
  36: 91,
  37: 93,
  38: 94,
  39: 96,
  40: 97,
  41: 99,
  42: 101,
  43: 104,
  44: 114,
  45: 116,
  46: 118,
  47: 121,
  48: 124,
  49: 135,
  50: 141,
  51: 147,
  52: 148,
  53: 149,
  54: 150,
  55: 152,
  56: 154,
  57: 155,
  58: 164,
  59: 166,
  60: 168,
  61: 187,
  62: 192,
  63: 199,
  64: 200,
  65: 209,
  66: 215,
  67: 217,
  68: 219,
  69: 226,
  70: 227,
  71: 228,
  72: 231,
  73: 234,
  74: 236,
  75: 237,
  76: 243,
  77: 251,
  78: 253,
  79: 255,
  80: 263,
  81: 264,
  82: 278,
  83: 279,
  84: 281,
  85: 284,
  86: 285,
  87: 286,
  88: 287,
  89: 288,
  90: 292,
  91: 299,
  92: 300,
  93: 303,
  94: 304,

In [28]:
pickle.dump(power_list.to_dict(),open('kpower_dict.pkl','wb'))

In [29]:
pickle.dump(power_pivot,open('power_pivot.pkl','wb'))

In [30]:
pickle.dump(power_model,open('power_model.pkl','wb'))

In [31]:
power_list['tempo']

0      123.047
1      136.014
2      127.997
3      129.961
4      128.008
        ...   
203    124.007
204    124.203
205    139.985
206    132.040
207    127.987
Name: tempo, Length: 208, dtype: float64